# This is an Notebook which can evalute multiple files.  It implements a number of features:

- TopK
- Removing recent referrals
- Thresholding 
- Tables and visualization
- Config files for standardized processes. 



In [5]:
%reload_ext autoreload
%autoreload 2
#ignore warnings.
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

#imports
import pandas as pd
import os, sys
import importlib
import yaml, json
from pathlib import Path

#path append
sys.path.append(os.path.join(Path.cwd(), 'modules'))
import Evaluate, Helper, Present, Score, Synthetic


<H3> Imports</H3>

### File configuration
<p> Just edit the configuration file path/address. prediction_files contain configs for different prediction files. config_file contains other configs for referral, visualization, experiment etc. </p>

In [6]:
#multiple prediction file load still works, just loaded a single file because too many things are being displayed otherwise.
config_file = 'config/config.yaml'
#prediction_config_files = ['config/predictions/pred1.yaml', 'config/predictions/pred2.yaml']
prediction_config_files = ['config/predictions/pred1.yaml']
generate_data=False 


### Load Configuration and Referrals

In [7]:
'''
Transferred referral data loading inside the c_p for loop below, because referral_data is being changed for date format adjustment. 
As referral_data.copy() is not allowed for memory issues. The overhead shouldn't be much.
'''

#Load Configuration
c_r, c_e, c_gen, c_aws, c_visual, predictions=Helper.load_configuration(config_file, prediction_config_files)

#Generate Data (if required)
if generate_data:
    Synthetic.generate_synthetic_event_data(c_gen)
    Synthetic.generate_synthetic_prediction_data(c_gen)

### Evaluate Models
For each configuration file provided it will evaluate the models. 

In [8]:
for c_p in predictions:    
    prediction=Helper.read_file(directory=c_p['dir'],file=c_p['file'],file_format=c_p['file_format'],aws=c_p['aws'],bucket= c_p['bucket'])
    referrals=Helper.read_file(directory=c_r['dir'],file=c_r['file'],file_format=c_r['file_format'],aws=c_r['aws'],bucket= c_r['bucket'])
    
    #eval date extract
    date_list = Helper.eval_date_extract(c_p, prediction)
    
    #run evaluation for all eval dates
    for eval_date in date_list:
        c_e['eval_date'] = eval_date
        all_model_evaluations = Evaluate.evaluate(c_p, c_e, c_r, referrals, prediction) 
        Present.present_evaluation(c_p, c_r, c_e, c_visual, eval_date, all_model_evaluations)